In [1]:
import os 

path = os.getcwd()
# find the string 'project' in the path, return index
index_project = path.find('project')
# slice the path from the index of 'project' to the end
project_path = path[:index_project+7]
# set the working directory
os.chdir(project_path)
print(f'Project path set to: {os.getcwd()}')


Project path set to: c:\Github\ode-biomarker-project


In [ ]:
import toolkit as tl 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# set the seaborn theme
sns.set_theme(style="whitegrid")
# set to talk 
sns.set_context("talk")